In [ ]:
!pip install numpy


In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight

In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight

In [4]:
# Load dataset (update the path to your dataset)
df = pd.read_csv('/content/community_health_evaluation_dataset.csv')
df

,Participant ID,Age,Gender,SES,Service Type,Visit Frequency,Step Frequency (steps/min),Stride Length (m),Joint Angle (°),EMG Activity,Patient Satisfaction (1-10),Quality of Life Score
0,1,56,F,4,Rehab,Weekly,85,0.54,17.99,Low,1,57
1,2,69,M,1,Preventive,Yearly,80,0.70,13.05,Moderate,8,94
2,3,46,M,4,Rehab,Yearly,81,0.57,29.85,Moderate,4,66
3,4,32,F,1,Consultation,Weekly,66,0.78,28.54,Moderate,9,66
4,5,60,M,3,Preventive,Weekly,73,0.84,20.80,High,5,98
...,...,...,...,...,...,...,...,...,...,...,...,...
342,343,24,M,3,Preventive,Yearly,92,0.67,27.45,Low,3,58
343,344,20,F,3,Rehab,Monthly,72,0.83,26.84,Low,4,81
344,345,34,F,3,Preventive,Yearly,89,0.90,13.72,Low,7,60
345,346,50,M,3,Preventive,Weekly,92,0.96,26.05,Moderate,3,57


In [5]:
df.isnull().sum()

,0
Participant ID,0
Age,0
Gender,0
SES,0
Service Type,0
Visit Frequency,0
Step Frequency (steps/min),0
Stride Length (m),0
Joint Angle (°),0
EMG Activity,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Participant ID               347 non-null    int64  
 1   Age                          347 non-null    int64  
 2   Gender                       347 non-null    object 
 3   SES                          347 non-null    int64  
 4   Service Type                 347 non-null    object 
 5   Visit Frequency              347 non-null    object 
 6   Step Frequency (steps/min)   347 non-null    int64  
 7   Stride Length (m)            347 non-null    float64
 8   Joint Angle (°)              347 non-null    float64
 9   EMG Activity                 347 non-null    object 
 10  Patient Satisfaction (1-10)  347 non-null    int64  
 11  Quality of Life Score        347 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 32.7+ KB


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
label_encoder = LabelEncoder()

# Fit and transform the data
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [10]:
label_encoder = LabelEncoder()

# Fit and transform the data
df['Service Type'] = label_encoder.fit_transform(df['Service Type'])

In [11]:
label_encoder = LabelEncoder()

# Fit and transform the data
df['Visit Frequency'] = label_encoder.fit_transform(df['Visit Frequency'])

In [12]:
label_encoder = LabelEncoder()

# Fit and transform the data
df['EMG Activity'] = label_encoder.fit_transform(df['EMG Activity'])

In [13]:
df.head()

,Participant ID,Age,Gender,SES,Service Type,Visit Frequency,Step Frequency (steps/min),Stride Length (m),Joint Angle (°),EMG Activity,Patient Satisfaction (1-10),Quality of Life Score
0,1,56,0,4,2,1,85,0.54,17.99,1,1,57
1,2,69,1,1,1,2,80,0.70,13.05,2,8,94
2,3,46,1,4,2,2,81,0.57,29.85,2,4,66
3,4,32,0,1,0,1,66,0.78,28.54,2,9,66
4,5,60,1,3,1,1,73,0.84,20.80,0,5,98


In [14]:
# Split the dataset into features and target variable
X = df.drop('EMG Activity', axis=1)
y = df['EMG Activity']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from collections import Counter

In [16]:
# Random Under Sampling
print("Original class distribution:", Counter(y_train))
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)
print("Original class distribution:", Counter(y_rus))

Original class distribution: Counter({2: 100, 0: 93, 1: 84})
Original class distribution: Counter({0: 84, 1: 84, 2: 84})


In [17]:
# Random Over Sampling
from imblearn.over_sampling import RandomOverSampler
print("Original class distribution:", Counter(y_train))
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
print("Original class distribution:", Counter(y_ros))

Original class distribution: Counter({2: 100, 0: 93, 1: 84})
Original class distribution: Counter({2: 100, 1: 100, 0: 100})


In [18]:
# SMOTE(Synthetic Minority Oversampling Techniqu)
print("Original class distribution:", Counter(y_train))
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)
print("Original class distribution:", Counter(y_smote))

Original class distribution: Counter({2: 100, 0: 93, 1: 84})
Original class distribution: Counter({2: 100, 1: 100, 0: 100})


In [19]:
# Tomek Links
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(X_train, y_train)

In [20]:
# Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))
class_weights

array([0.99283154, 1.09920635, 0.92333333])

In [21]:
# Function to evaluate model performance
def evaluate_model(X_train, y_train, X_test, y_test, class_weights=None):
    model = RandomForestClassifier(class_weight=class_weights, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get probabilities for each class
    y_prob = model.predict_proba(X_test)

    # Print classification report
    print(classification_report(y_test, y_pred))

    # Calculate AUC for multi-class
    y_test_binarized = label_binarize(y_test, classes=model.classes_)
    auc = roc_auc_score(y_test_binarized, y_prob, average="macro", multi_class="ovr")
    print("AUC:", auc)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize

# Evaluate each sampling technique
print("Random Undersampling:")
evaluate_model(X_rus, y_rus, X_test, y_test)

print("\nRandom Oversampling:")
evaluate_model(X_ros, y_ros, X_test, y_test)

print("\nSMOTE:")
evaluate_model(X_smote, y_smote, X_test, y_test)

#print("\nTomek Links:")
#evaluate_model(X_tl, y_tl, X_test, y_test)

print("\nClass Weights:")
evaluate_model(X_train, y_train, X_test, y_test, class_weights_dict)

Random Undersampling:
              precision    recall  f1-score   support

           0       0.26      0.29      0.27        24
           1       0.28      0.20      0.23        25
           2       0.32      0.38      0.35        21

    accuracy                           0.29        70
   macro avg       0.29      0.29      0.28        70
weighted avg       0.28      0.29      0.28        70

AUC: 0.46258062797237604

Random Oversampling:
              precision    recall  f1-score   support

           0       0.40      0.58      0.47        24
           1       0.46      0.24      0.32        25
           2       0.45      0.48      0.47        21

    accuracy                           0.43        70
   macro avg       0.44      0.43      0.42        70
weighted avg       0.44      0.43      0.42        70

AUC: 0.4973107608813021

SMOTE:
              precision    recall  f1-score   support

           0       0.32      0.33      0.33        24
           1       0.38     